<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/data_postprocess/prediction_postprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import numpy as np
import pandas as pd

#from google.colab import data_table
#data_table.enable_dataframe_formatter()

In [3]:
# Some important file locations and constants

project_root = "/content/drive/MyDrive/w266 NLP Final Project/"
prediction_folder = project_root + "Predictions/checkpoint/"
data_folder = project_root + "Data/"
processed_folder = project_root + "Predictions/"

datasets = ['nq', 'quac', 'squad.hf', 'triviaqa']

validation_set_size = {}

In [4]:
for dataset in datasets:
  df = pd.read_csv(f"{data_folder}{dataset}/valid_pairs.csv")
  validation_set_size [dataset.split(".")[0]] = df.shape[0]

In [5]:
validation_set_size

{'nq': 2356, 'quac': 5868, 'squad': 10570, 'triviaqa': 9835}

In [6]:
file_list = glob.glob(prediction_folder + "*.beams.csv")

In [7]:
file_list

['/content/drive/MyDrive/w266 NLP Final Project/Predictions/checkpoint/predictions.bart_base_pt_long.nq.squad.beams.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions/checkpoint/predictions.bart_base_pt_long.quac.triviaqa.beams.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions/checkpoint/predictions.bart_base_pt_long.squad.quac.beams.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions/checkpoint/predictions.bart_base_pt_long.squad.nq.beams.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions/checkpoint/predictions.bart_base_pt_long.quac.quac.beams.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions/checkpoint/predictions.bart_base_pt_long.triviaqa.triviaqa.beams.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions/checkpoint/predictions.bart_base_pt_long.quac.nq.beams.csv',
 '/content/drive/MyDrive/w266 NLP Final Project/Predictions/checkpoint/predictions.bart_base_pt_long.triviaqa.nq.beams.csv',
 '/co

In [8]:
len(file_list)

20

In [9]:
df = pd.DataFrame([(os.path.basename(x), len (pd.read_csv(x))) for x in file_list], columns=['name', 'count']).sort_values(by=['count'])
df

,name,count
17,predictions.bart_base_pt_long.amalgam.nq.beams...,2356
3,predictions.bart_base_pt_long.squad.nq.beams.csv,2356
8,predictions.bart_base_pt_long.nq.nq.beams.csv,2356
7,predictions.bart_base_pt_long.triviaqa.nq.beam...,2356
6,predictions.bart_base_pt_long.quac.nq.beams.csv,2356
9,predictions.bart_base_pt_long.triviaqa.quac.be...,5868
11,predictions.bart_base_pt_long.nq.quac.beams.csv,5868
19,predictions.bart_base_pt_long.amalgam.quac.bea...,5868
4,predictions.bart_base_pt_long.quac.quac.beams.csv,5868
2,predictions.bart_base_pt_long.squad.quac.beams...,5868


In [10]:
for name in file_list:
  df = pd.read_csv(name)
  df = df.replace(r'^\s*$', np.nan, regex=True)
  count = df['prediction'].isna().sum()
  if count > 0:
    print (f"{os.path.basename(name)} empty predictions: {count}")

In [11]:
for file in file_list:
  model, training, validation = os.path.basename(file).split(".")[1:4]
  df = pd.read_csv(file)
  print (f"{model} {training} validation {validation}", end=" ")
  if df.shape[0]==validation_set_size[validation]:
    print ("proper length")
  else:
    print ("BAD LENGTH")  

bart_base_pt_long nq validation squad proper length
bart_base_pt_long quac validation triviaqa proper length
bart_base_pt_long squad validation quac proper length
bart_base_pt_long squad validation nq proper length
bart_base_pt_long quac validation quac proper length
bart_base_pt_long triviaqa validation triviaqa proper length
bart_base_pt_long quac validation nq proper length
bart_base_pt_long triviaqa validation nq proper length
bart_base_pt_long nq validation nq proper length
bart_base_pt_long triviaqa validation quac proper length
bart_base_pt_long squad validation triviaqa proper length
bart_base_pt_long nq validation quac proper length
bart_base_pt_long nq validation triviaqa proper length
bart_base_pt_long triviaqa validation squad proper length
bart_base_pt_long squad validation squad proper length
bart_base_pt_long amalgam validation squad proper length
bart_base_pt_long quac validation squad proper length
bart_base_pt_long amalgam validation nq proper length
bart_base_pt_long

In [12]:
for file in file_list:
  df = pd.read_csv(file, usecols=['context', 'answer', 'target', 'prediction'])
  df = df.replace(r'^\s*$', np.nan, regex=True)
  df['prediction'] = df['prediction'].fillna('NO PREDICTION MADE')
  output_file = f"{processed_folder}{os.path.basename(file)}"
  df.to_csv(output_file, index=False)

In [13]:
for file in file_list:
  model, training, validation = os.path.basename(file).split(".")[1:4]
  test_file = f"{processed_folder}{os.path.basename(file)}"
  df = pd.read_csv(test_file)
  print(df.keys())

Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'target', 'prediction'], dtype='object')
Index(['context', 'answer', 'targe

In [15]:
for file in file_list:
  model, training, validation = os.path.basename(file).split(".")[1:4]
  test_file = f"{processed_folder}{os.path.basename(file)}"
  df = pd.read_csv(test_file)
  df = df[df['prediction'] == "NO PREDICTION MADE"]
  if df.shape[0]: 
    print (test_file, df.shape[0])
    print (df.index)